# Analyze variation within specific GCFs

Here we summarize workflow used to study the variations within particular Gene Cluster Families (GCFs) of plipastatin, fengycin and iturinic lipopetides

### Data directories used in this analysis

    1. ../data/antismash_out : Output folders for all genomes in analysis also used as input for BiGSCAPE software. This is not included in the Github directory, and users can adapt this locally. We use this data only to extract amino acid specificity in this analysis, which part should be adapted accordingly.
    2. ../data/bigscape : Output of BiGSCAPE dataset, which will be used to create network and detect families
    3. ../data/cytoscape : To save tables to be used by Cytoscape for visualization
    4. ../data/bigscape_selected : Rerunning BiGSCAPE for subgroups within specific GCFs to generate CORASSON alignments
    5. ../data/frameshift : Nucleotide sequence alignments for selected genes to show conserved frameshift mutations
    6. ../tables : Directory to save tables with information on BGCs 

In [ ]:
# Import libraries
import os
from Bio import SeqIO
import pandas as pd
import networkx as nx
from shutil import copyfile
from Bio.Align.Applications import MuscleCommandline
from collections import OrderedDict

### Start from the BiGSCAPE output

BiG-SCAPE output is stored in '../data/bigscape' directory. Following analysis reads the BiGSCAPE outpur directory and the run labelled as '2020-09-26_08-30-48_hybrids_glocal'

In [ ]:
path_dict = dict(bigscape_path = '../data/bigscape/')

In [ ]:
def update_path_bigscape(path_dict, run_label):
    '''
    Update path dictionary with tables from BiG-SCAPE analysis 
    '''
    
    networks_dir = os.path.join(path_dict['bigscape_path'], 'network_files')
    run_name = [folder for folder in os.listdir(networks_dir) if run_label in folder][0]
    path_dict['net_dir'] = os.path.join(networks_dir, run_name)
    path_dict['node_table'] = os.path.join(networks_dir, run_name, 'Network_Annotations_Full.tsv')
    path_dict['cyto_inputs'] = '../data/cytoscape/input/'
    
    return path_dict


def combine_networks(path_dict, cut_off, run_label):
    '''
    Combines networks of different classes to generate single network file
    '''

    net_dir = path_dict['net_dir']
    cluster_classes = [cluster_class for cluster_class in os.listdir(net_dir) if not 'Network_Annotations_Full' in cluster_class]

    df_network_union = pd.DataFrame()
    
    for cluster_class in cluster_classes:
        class_dir = os.path.join(net_dir, cluster_class)
        net_file = cluster_class + '_c' + cut_off + '.network'
        
        if os.path.isfile(os.path.join(class_dir, net_file)):
            df_tmp = pd.read_csv(os.path.join(class_dir, net_file), sep='\t')
            df_network_union = df_network_union.append(df_tmp, ignore_index = True) 

    cyto_inputs = path_dict['cyto_inputs']
    filename = os.path.join(cyto_inputs, run_label + '_network_' + cut_off + '.csv')
    df_network_union.to_csv(filename)
    
    return df_network_union


def get_node_table(path_dict, run_label):
    '''
    Update node annotations for cytoscape
    '''
    
    node_table = path_dict['node_table']
    
    df_nodes = pd.read_csv(node_table, sep='\t')
    df_nodes.set_index('BGC',inplace=True)
   
    for bgc in df_nodes.index:
        if 'BGC0' in bgc:
            mibig_name = df_nodes.loc[bgc, 'Description'].split(' biosynthetic gene cluster')[0]
            df_nodes.loc[bgc, 'genome_id'] = mibig_name
            df_nodes.loc[bgc, 'genome_src'] = 'MIBIG'
            df_nodes.loc[bgc, 'species'] = 'MIBIG'
            org = df_nodes.loc[bgc, 'Organism']
            if len(org.split(' ')) > 1:
                species = org.split(' ')[1]
                df_nodes.loc[bgc, 'species'] = species
        else:
            genome_id = df_nodes.loc[bgc, 'Accesion ID'] 
            df_nodes.loc[bgc, 'genome_id'] = ''
            df_nodes.loc[bgc, 'genome_src'] = 'Public'
            org = df_nodes.loc[bgc, 'Organism']
            species = org.split(' ')[1]
            df_nodes.loc[bgc, 'species'] = species
            
    cyto_inputs = path_dict['cyto_inputs']
    filename = os.path.join(cyto_inputs, run_label + '_nodes.csv')
    df_nodes.to_csv(filename)
    
    return df_nodes
    
    
cut_off = '0.30'
run_label = '2020-09-26_08-30-48_hybrids_glocal'

path_dict = update_path_bigscape(path_dict, run_label)
df_network_union = combine_networks(path_dict, cut_off, run_label)
df_nodes = get_node_table(path_dict, run_label)

In [ ]:
# Use networkx to generate a graph
G_clusters = nx.from_pandas_edgelist(df_network_union, 'Clustername 1', 'Clustername 2', 'Raw distance')

In [ ]:
# Find connected components or BGC families
Families_list = list(nx.connected_component_subgraphs(G_clusters))
print('Total families detected: ', len(Families_list))

# Sort families in decreasing order of size
family_size = [len(family) for family in Families_list]
orig_index = list(range(len(family_size)))
orig_index_fam_size_dict = dict(zip(orig_index, family_size))

sorted_index_fam_size_dict = OrderedDict(sorted(orig_index_fam_size_dict.items(), key=lambda x: x[1]), )
new_index = list(range(len(family_size)-1,-1,-1))
orig_index_sorted = list(sorted_index_fam_size_dict.keys())
new_orig_index_dict = dict(zip(new_index, orig_index_sorted))

# Ordered family graphs
family_graphs = [Families_list[new_orig_index_dict[fam_id]] for fam_id in range(len(Families_list))]

### Visualization of network and Cytoscape and intermediate analysis

The df_network and df_nodes are stored in '../data/cytoscape/input/' directory. These tables are then imported in Cytoscape for visualization of network. Included in Figure S2


All the nodes belonging to selected GCFs are then stored in '../tables/df_clusters.csv'. In this study, we isolated BGCs belonging to plipastatin, fengycin GCFs. The classification of these BGCs into subgroups was carried out based on network structure in Cytoscape, which is a manual step. Following steps use this classification from '../tables/df_clusters.csv'.

### Add PATRIC information Geographic location

Location information was collected using metadata available at PATRIC database. PATRIC table was downloaded and stored at '../tables/PATRIC_genomes.csv'

In [ ]:
df_patric = pd.read_csv('../tables/PATRIC_genomes.csv', dtype={'Genome ID':str, 'RefSeq Accessions': str}, engine='python')
df_clusters = pd.read_csv('../tables/df_clusters.csv', index_col='name')
df_patric.set_index('Genome ID', inplace=True)
df_patric.fillna('',inplace=True)

In [ ]:
gbk_dict = dict()
refseq_dict = dict()
for patric_id in df_patric.index:
    gbk_accn = df_patric.loc[patric_id, 'GenBank Accessions']
    if ',' in gbk_accn:
        gbk_list = gbk_accn.split(',')
        for gbk_iter in gbk_list:
            if '.' in gbk_iter:
                gbk_id = gbk_iter.split('.')[0]
            else:
                gbk_id = gbk_iter
            gbk_dict[gbk_id] = patric_id
    else:
        if '.' in gbk_accn:
            gbk_id = gbk_accn.split('.')[0]
        else:
            gbk_id = gbk_accn
       
        gbk_dict[gbk_id] = patric_id
    
    refseq_accn = df_patric.loc[patric_id, 'RefSeq Accessions']
    
    if ',' in refseq_accn:
        refseq_list = refseq_accn.split(',')
        for refseq_iter in refseq_list:
            if '.' in refseq_iter:
                refseq_id = refseq_iter.split('.')[0]
            else:
                refseq_id = refseq_iter
            refseq_dict[refseq_id] = patric_id
    else:
        if '.' in refseq_accn:
            refseq_id = refseq_accn.split('.')[0]
        else:
            refseq_id = refseq_accn
        refseq_dict[refseq_id] = patric_id
    
for bgc_id in df_clusters.index:
    accn = df_clusters.loc[bgc_id, 'Accesion ID']
    if '_' in accn:
        if '.' in accn:
            gbk_id = accn.split('_')[1].split('.')[0]
        else:
            gbk_id = accn.split('_')[1]
    elif '.' in accn:
        gbk_id = accn.split('.')[0]

    df_clusters.loc[bgc_id, 'Genbank Accession'] = gbk_id
    if gbk_id in gbk_dict.keys():
        df_clusters.loc[bgc_id, 'PATRIC'] = gbk_dict[gbk_id]
    else:
        if accn in refseq_dict.keys():
            df_clusters.loc[bgc_id, 'PATRIC'] = refseq_dict[accn]
        elif accn[:-2] in refseq_dict.keys():
            df_clusters.loc[bgc_id, 'PATRIC'] = refseq_dict[accn[:-2]]
        else:
            print(accn, df_clusters.loc[bgc_id, 'Organism'])
    
for bgc_id in df_clusters.index:
    patric_id = df_clusters.loc[bgc_id, 'PATRIC']
    if patric_id in df_patric.index:
        df_clusters.loc[bgc_id, 'Location'] = df_patric.loc[patric_id, 'Geographic Location']

### Analyze Plipastatin family in details

In [ ]:
pps_groups = ['PPS', 'PPS_groupB', 'PPS_groupC', 'PPS_groupD', 'PPS_groupE', 'PPS_others']
df_plipastatins = df_clusters[df_clusters.Groups.isin(pps_groups)]
df_plipastatins = df_plipastatins.sort_values(by=['Groups', 'Clan Number','Family Number'])

In [ ]:
df_clusters.head()

In [ ]:
# This step required all antiSMASH processed data which is not shared on Github.
# This step can be adapted as per the local user
for node in df_plipastatins.index:
    if 'BGC' not in node:
        genome_id = df_plipastatins.loc[node, 'Accesion ID']
        gbk_path = os.path.join('../data/antismash_out/', genome_id, node + '.gbk')
        if not os.path.isfile(gbk_path):
            gbk_path = os.path.join('../data/antismash_out/', genome_id.split('.')[0], node + '.gbk')
        AA_list = []
        with open(gbk_path) as in_handle:
            for rec in SeqIO.parse(in_handle, 'genbank'):
                for feat in rec.features:
                    if feat.type == 'aSDomain':
                        if 'specificity' in feat.qualifiers.keys():
                            AA_specificity = feat.qualifiers['specificity'][0].split(': ')[1]
                            AA_list.append(AA_specificity)
        df_plipastatins.loc[node,'AA_domain_list'] = ','.join(AA_list)

In [ ]:
df_plipastatins = df_plipastatins[['Accesion ID', 'Organism', 'species', 'PATRIC', 'Location', 'Product Prediction', 
                                   'BiG-SCAPE class', 'Family Number', 'Clan Number', 'Groups', 'AA_domain_list']]

In [ ]:
df_plipastatins.to_csv('../tables/df_plipastatins.csv')

### Analyze fengycin family in details

In [ ]:
fen_groups = ['FEN', 'FEN_groupB', 'FEN_groupC', 'FEN_Others']
df_fengycins = df_clusters[df_clusters.Groups.isin(fen_groups)]
df_fengycins = df_fengycins.sort_values(by=['Groups', 'Clan Number','Family Number'])

In [ ]:
# This step required all antiSMASH processed data which is not shared on Github.
# This step can be adapted as per the local user
for node in df_fengycins.index:
    if 'BGC' not in node:
        genome_id = df_fengycins.loc[node, 'Accesion ID']
        gbk_path = os.path.join('../data/antismash_out/', genome_id, node + '.gbk')
        if not os.path.isfile(gbk_path):
            gbk_path = os.path.join('../data/antismash_out/', genome_id.split('.')[0], node + '.gbk')
        AA_list = []
        with open(gbk_path) as in_handle:
            for rec in SeqIO.parse(in_handle, 'genbank'):
                for feat in rec.features:
                    if feat.type == 'aSDomain':
                        if 'specificity' in feat.qualifiers.keys():
                            AA_specificity = feat.qualifiers['specificity'][0].split(': ')[1]
                            AA_list.append(AA_specificity)
        df_fengycins.loc[node,'AA_domain_list'] = ','.join(AA_list)

In [ ]:
df_fengycins = df_fengycins[['Accesion ID', 'Organism', 'species', 'PATRIC', 'Location', 'Product Prediction', 
                                   'BiG-SCAPE class', 'Family Number', 'Clan Number', 'Groups', 'AA_domain_list']]
df_fengycins.to_csv('../tables/df_fengycins.csv')

### Analyze Iturins family in details

Apart from Plipastatin and Fengycin GCFs, we further analyzed iturinic lipopeptide GCF in the following section.

In [ ]:
iturin_myco_bacillo_fam = family_graphs[7] # 8th largest family including iturins

### Define iturinic AA specificty 

In [ ]:
df_iturins = df_nodes.copy()
df_iturins = df_iturins.loc[list(iturin_myco_bacillo_fam.nodes), :]

iturin_aa = ['ser','asn','pro','gln','asn','tyr','asn']
iturin_fw = ','.join(iturin_aa)
iturin_bw = ','.join(iturin_aa[::-1])

bacillomycinD_aa = ['thr','ser','glu','pro','asn','tyr','asn']
bacillomycinD_fw = ','.join(bacillomycinD_aa)
bacillomycinD_bw = ','.join(bacillomycinD_aa[::-1])

bacillomycinL_aa = ['thr','ser','glu','ser','asn','tyr','asn']
bacillomycinL_fw = ','.join(bacillomycinL_aa)
bacillomycinL_bw = ','.join(bacillomycinL_aa[::-1])

bacillomycinL_aa = ['thr','ser','glu','ser','asn','tyr','asn']
bacillomycinL_fw = ','.join(bacillomycinL_aa)
bacillomycinL_bw = ','.join(bacillomycinL_aa[::-1])

bacillomycinF_aa = ['thr','asn','pro','gln','asn','tyr','asn']
bacillomycinF_fw = ','.join(bacillomycinF_aa)
bacillomycinF_bw = ','.join(bacillomycinF_aa[::-1])

mycosubtilin_aa = ['asn','ser','pro','gln','asn','tyr','asn']
mycosubtilin_fw = ','.join(mycosubtilin_aa)
mycosubtilin_bw = ','.join(mycosubtilin_aa[::-1])

In [ ]:
# This step required all antiSMASH processed data which is not shared on Github.
# This step can be adapted as per the local user
for node in df_iturins.index:
    if 'BGC' not in node:
        genome_id = df_iturins.loc[node, 'Accesion ID']
        gbk_path = os.path.join('../data/antismash_out/', genome_id, node + '.gbk')
        if not os.path.isfile(gbk_path):
            gbk_path = os.path.join('../data/antismash_out/', genome_id.split('.')[0], node + '.gbk')
        AA_list = []
        with open(gbk_path) as in_handle:
            for rec in SeqIO.parse(in_handle, 'genbank'):
                for feat in rec.features:
                    if feat.type == 'aSDomain':
                        if 'specificity' in feat.qualifiers.keys():
                            AA_specificity = feat.qualifiers['specificity'][0].split(': ')[1]
                            AA_list.append(AA_specificity)
        AA_str = ','.join(AA_list)
        df_iturins.loc[node,'AA_domain_list'] = AA_str
        df_iturins.loc[node,'family_number'] = df_nrps_fam_id.loc[node, 'Family Number']
        df_iturins.loc[node,'clan_number'] = df_nrps_fam_id.loc[node, 'Clan Number']
        
        df_iturins.loc[node, 'Groups'] = 'Itu_Others'
        
        if iturin_fw in AA_str or iturin_bw in AA_str:
            df_iturins.loc[node, 'Groups'] = 'Iturin A'
           
        if bacillomycinD_fw in AA_str or bacillomycinD_bw in AA_str:
            df_iturins.loc[node, 'Groups'] = 'Bacillomycin D'
            
        if bacillomycinL_fw in AA_str or bacillomycinL_bw in AA_str:
            df_iturins.loc[node, 'Groups'] = 'Bacillomycin L'
            
        if bacillomycinF_fw in AA_str or bacillomycinF_bw in AA_str:
            df_iturins.loc[node, 'Groups'] = 'Bacillomycin F'
            
        if mycosubtilin_fw in AA_str or mycosubtilin_bw in AA_str:
            df_iturins.loc[node, 'Groups'] = 'Mycosubtilin'
        

In [ ]:
df_iturins.to_csv('../tables/df_iturins.csv')

### Update df_clusters with AA specificity 

In [ ]:
df_clusters.loc[df_plipastatins.index, 'AA_domain_list'] = df_plipastatins.AA_domain_list
df_clusters.loc[df_fengycins.index, 'AA_domain_list'] = df_fengycins.AA_domain_list

for col in df_clusters.columns:
    if col in df_iturins.columns:
        for bgc_id in df_iturins.index:
            df_clusters.loc[bgc_id, col] = df_iturins.loc[bgc_id, col]

In [ ]:
df_AA_species = pd.DataFrame(index=df_clusters.Groups.unique(), columns=['AA_domain_list', 
                    'velezensis', 'amyloliquefaciens', 'subtilis', 'atrophaeus', 'sp.'])

df_AA_species[['velezensis', 'amyloliquefaciens', 'subtilis', 'atrophaeus', 'sp.']] = 0
for node in df_clusters.index:
    if 'BGC0' not in node:
        predict_fam = df_clusters.loc[node, 'Groups']
        AA_domain_list = df_clusters.loc[node, 'AA_domain_list']
        species = df_clusters.loc[node, 'species']

        df_AA_species.loc[predict_fam, species] = df_AA_species.loc[predict_fam, species] + 1
        df_AA_species.loc[predict_fam, 'AA_domain_list'] = AA_domain_list    

In [ ]:
df_AA_species # Table S1

### Generate matrix for the Phylogentic tree input 

Phylogenetic tree and mapping of gene presence absence matrix was carried out as per the https://github.com/KatSteinke/AbsPresTree and included in Figure 2 and Figure S4

In [ ]:
accn_list = os.listdir('../data/antismash_out/')
col_fams = list(df_clusters.Groups.unique())
df_subfam_mat = pd.DataFrame(0, index=accn_list, columns=col_fams)

for bgc_id in df_clusters.index:
    if 'BGC0' not in bgc_id:
        accn_id = df_clusters.loc[bgc_id, 'Accesion ID']
        subfam = df_clusters.loc[bgc_id, 'Groups']
        df_subfam_mat.loc[accn_id, subfam] = 1
        if accn_id not in accn_list:
            print(accn_id)

df_subfam_mat.fillna(0, inplace=True)
df_subfam_mat.sort_index(axis=1, inplace=True)
df_subfam_mat.to_csv('../tables/subfam_dist.csv')

In [ ]:
sns.clustermap(df_subfam_mat, cmap='BuPu')

## Analyze selected clusters separately using cluster alignments

BiGSCAPE and CORASSON analysis was repeated separately on individual families of plipastatin and fengycin GCFs, to create Figure S3

In [ ]:
df_clusters = pd.read_csv('../tables/df_clusters.csv', index_col='name')
selected_clusters_path = '../data/bigscape_selected/'

In [ ]:
# Create directory with selected clusters
groups_selected = ['FEN', 'PPS_groupD', 'PPS_groupE', 'PPS', 'FEN_groupB', 'FEN_groupC', 'PPS_groupC', 'FEN_Others', 'PPS_groupB', 'PPS_others']

anti_path = '../data/antismash_out/'
for group_name in groups_selected:
    df_clusters_group = df_clusters[df_clusters.Groups == group_name]
    group_dir = os.path.join(selected_clusters_path, group_name)
    if not os.path.isdir(group_dir):
        os.mkdir(group_dir)
        os.mkdir(os.path.join(group_dir, 'input'))
        os.mkdir(os.path.join(group_dir, 'output'))
    for cluster_id in df_clusters_group.index:
        genome_id = df_clusters_group.loc[cluster_id, 'Accesion ID']
        if os.path.isfile(os.path.join(anti_path, genome_id, cluster_id + '.gbk')):
            from_path = os.path.join(anti_path, genome_id, cluster_id + '.gbk')
            to_path = os.path.join(group_dir, 'input', cluster_id + '.gbk')
            copyfile(from_path, to_path)
        else:
            print(cluster_id, 'not found') 
    from_path = os.path.join('../data/bigscape_selected/Fengycin.region.gbk')
    to_path = os.path.join(group_dir, 'input', 'Fengycin.region.gbk')
    copyfile(from_path, to_path)
    from_path = os.path.join('../data/bigscape_selected/Plipastatin.region.gbk')
    to_path = os.path.join(group_dir, 'input', 'Plipastatin.region.gbk')
    copyfile(from_path, to_path)

In [ ]:
# Run BiGSCAPE on each of the group
for group_name in groups_selected:
    input_dir = os.path.join(selected_clusters_path, group_name, 'input')
    out_dir = os.path.join(selected_clusters_path, group_name, 'output')
    bigscape_path = '/home/omkar/Projects/packages/bigscape5/BiG-SCAPE/bigscape.py'
    
    cmd = 'python ' + bigscape_path + ' -i ' + input_dir + ' -o ' + out_dir + ' --cutoff 0.3 0.5 0.7 --include_singletons'
    print(cmd)
    os.system(cmd)

### Compare genes with frameshifts

### Case 1: ppsE gene

In [ ]:
df_clusters = pd.read_csv('../tables/df_clusters.csv', index_col='name')
df_selected = df_clusters[df_clusters.Groups == 'PPS_groupE']
gbk_path = '../data/bigscape_selected/PPS_groupE/input/'
selected_protein_id = 'NP_389712.1'
multi_fasta_out = '../data/frameshift/ppsE_groupE.faa'
align_out = '../data/frameshift/ppsE_groupE.align'
reference_bgc = '../data/bigscape_selected/Plipastatin.region.gbk'
reference_gene = 'ppsE'

### Case 2: fenD gene

In [ ]:
df_clusters = pd.read_csv('../tables/df_clusters.csv', index_col='name')
df_selected = df_clusters[df_clusters.Groups == 'FEN_groupB']
gbk_path = '../data/bigscape_selected/FEN_groupB/input/'
selected_protein_id = 'WP_014305115.1'
multi_fasta_out = '../data/frameshift/fenD_groupB.faa'
align_out = '../data/frameshift/fenD_groupB.align'
reference_bgc = '../data/bigscape_selected/Fengycin.region.gbk'
reference_gene = 'fenD'

In [ ]:
# Create multifasta file
selected_sequences = dict()
for rec in SeqIO.parse(reference_bgc, 'genbank'):
    for feat in rec.features:
        if feat.type == 'CDS':
            if 'gene' in feat.qualifiers:
                gene = feat.qualifiers['gene'][0]
                if gene == reference_gene:
                    seq_selected = feat.location.extract(rec).seq
                    selected_sequences[gene] = seq_selected

for bgc_id in df_selected.index:
    gbk_in = os.path.join(gbk_path, bgc_id + '.gbk')
    print(bgc_id)
    for rec in SeqIO.parse(gbk_in, 'genbank'):
        for feat in rec.features:
            if 'inference' in feat.qualifiers:
                inference = feat.qualifiers['inference'][0]
                if selected_protein_id in inference:
                    locus_tag_selected = feat.qualifiers['locus_tag'][0]
                    seq_selected = feat.location.extract(rec).seq
                    genome_id = df_clusters.loc[bgc_id, 'Accesion ID']
                    locus_tag_selected = genome_id + '_' + locus_tag_selected
                    print(locus_tag_selected)
                    selected_sequences[locus_tag_selected] = seq_selected
            if feat.type == 'CDS':
                if 'note' in feat.qualifiers:
                    note = feat.qualifiers['note'][0]
                    if 'frameshift' in note:
                        print(feat.qualifiers['inference'][0])
                
with open(multi_fasta_out, "w") as output_handle:
    for locus in selected_sequences.keys():
        seq = selected_sequences[locus]
        output_handle.write('>%s\n%s\n'%(locus, seq))

In [ ]:
# Align using MUSCLE
cline = MuscleCommandline(input=multi_fasta_out, out=align_out)
stdout, stderr = cline()